In [1]:
import MinCompSpin as mod
import math
import numpy as np

'''
Create a fully connected partitioning of r variables.
'''
def createPartition(r):
    if r <= 64:
        bitset = [0, (1 << r) - 1, 0]
    elif r <= 128:
        bitset = [0, (1 << 64) - 1, (1 << (r - 64)) - 1]
    else:
        raise Exception("r can not be greater than 128")
    partition = mod.Partitions(np.array([bitset], dtype=np.uint64), r)
    return partition

'''
Create a partition of size r with no variables in it.
'''
def createEmptyParition(r):
    bitset = [0, 0, 0]
    partition = mod.Partitions(np.array([bitset], dtype=np.uint64), r)
    return partition

def copyPartition(partitions):
    return mod.Partitions(partitions.array, partitions.r)

'''
Return true if the partition has no variables
!!! Only works on (partitions) objects with 1 partition.
'''
def partitionEmpty(partition):
    assert partition.array.shape[0] == 1
    return partition.array[0, 1] == 0 and partition.array[0, 2] == 0

'''
Return true if the partition contains the variable i.
'''
def partitionContains(partitionA, i):
    for partition in partitionA.array:
        index = i // 64 + 1
        offcet = np.uint64(i % 64)
        bitset = partition[index]
        if (bitset >> offcet) & np.uint64(1):
            return True
    return False

'''
Merge 2 partitions into one.
'''
def joinPartitions(partitionA, partitionB):
    newArray = []
    ids = 1
    if partitionA is not None:
        for partition in partitionA.array:
            newArray.append([ids, partition[1], partition[2]])
            ids += 1
    if partitionB is not None:
        for partition in partitionB.array:
            newArray.append([ids, partition[1], partition[2]])
            ids += 1
    return mod.Partitions(np.array(newArray), partitionA.r)

'''
Move the i'th variable from the 'fromPartition' to the 'toPartition.'
!!! Only works on (Partitions) objects with 1 partition.
'''
def swapVariable(fromPartition, toPartition, i):
    assert(fromPartition.array.shape[0] == 1 and toPartition.array.shape[0] == 1)
    index = i // 64 + 1
    offcet = i % 64
    # Verify that the bit is set in fromPartition
    if not (fromPartition.array[0, index] & np.uint64(1 << offcet)):
        print(index, offcet)
    assert fromPartition.array[0, index] & np.uint64(1 << offcet)
    # Unset the bit
    fromPartition.array[0, index] ^= np.uint64(1 << offcet)
    # Verify that the bit is NOT set in toPartition
    assert toPartition.array[0, index] & np.uint64(1 << offcet) == 0
    # Set the bit
    toPartition.array[0, index] ^= np.uint64(1 << offcet)

'''
Greedily optimize the distribution of variables over the two partitions.
'''
def optimizePair(iccA, iccB, oldLogE, Kset):
    r = iccA.r
    print('oP')
    history = [(oldLogE, None)]
    while not partitionEmpty(iccA):
        bestLogE = -math.inf
        best_i = None
        count = 0
        for i in range(r):
            if not partitionContains(iccA, i):
                continue
            count = i
            swapVariable(iccA, iccB, i)
            newLogE = mod.LogE_ICC(Kset, iccA) + mod.LogE_ICC(Kset, iccB)
            if math.isnan(newLogE) and best_i == None: # When an ICC is empty mod.LogE_ICCreturns NaN
                best_i = i
            elif (newLogE > bestLogE):
                bestLogE = newLogE
                best_i = i
            swapVariable(iccB, iccA, i) # swap back
        # best_i is only none when all variables have already been removed from partitionA
        # This shouldn't happen because we check that partitionA is not empty
        assert best_i is not None
        swapVariable(iccA, iccB, best_i)
        history.append((bestLogE, best_i))

    # Find the best partitioning in the history
    bestLogE = history[0][0]
    bestHistory = 0
    for i in range(1, len(history)):
        if history[i][0] > bestLogE:
            bestLogE = history[i][0]
            bestHistory = i
    print('best', history[bestHistory])
    # Roll back to the best version
    for i in range(len(history)-1, bestHistory, -1):
        swapVariable(iccB, iccA, history[i][1])
    return bestLogE

'''
Use a divide and conquer approach to find an optimial partitioning of the variables for an MCM with the data in Kset.
'''
def divideAndConquerSearch(Kset, iccIn = None):
    print("dACS")
    if iccIn is None:
        # Create partition with all variables
        iccIn = createPartition(Kset.r)
    if iccIn.r <= 1:
        return iccIn
    oldLogE = mod.LogE_ICC(Kset, iccIn)
    print("oldLogE:", oldLogE)
    # Create new partition pair
    iccA = copyPartition(iccIn)
    iccB = createEmptyParition(Kset.r)
    newLogE = optimizePair(iccA, iccB, oldLogE, Kset)
    if newLogE <= oldLogE:
        # No further partitioning
        print('found nothing')
        return iccIn
    else:
        # Divide and Conquer
        newPartitionA = divideAndConquerSearch(Kset, iccA)
        newPartitionB = divideAndConquerSearch(Kset, iccB)
        return joinPartitions(newPartitionA, newPartitionB)
    

In [6]:
# path, r = 'MinCompSpin_Greedy/INPUT/my_data_n60_N250.dat', 60      # Here the log evidence doesn't increase
# path, r = 'MinCompSpin_Greedy/INPUT/my_data_n100_N1000.dat', 100   # Here the log evidence calculation is wrong
path, r = 'MinCompSpin_Greedy/INPUT/SCOTUS_n9_N895_Data.dat', 9    # This dataset seems to work
Kset = mod.read_datafile(path, r)
result = divideAndConquerSearch(Kset)
finalLogE = mod.LogE_MCM(Kset, result)
assert finalLogE != 0
print("final LogE:", finalLogE)
mod.Print_MCM_Partition(result)

if True: # Only run greedy version when I'm testing that
    greedyResult = mod.GreedySearch(Kset)
    greedyLogE = mod.LogE_MCM(Kset, greedyResult)
    print("greedy LogE:", greedyLogE)
    mod.Print_MCM_Partition(greedyResult)

dACS
oldLogE: -3305.546575913684
oP
Read the dataset: MinCompSpin_Greedy/INPUT/SCOTUS_n9_N895_Data.dat
Number of variables to read: n = 9
best (-3300.395469673638, 3)
dACS
oldLogE: -1754.411660223669
oP
best (-1754.411660223669, None)
found nothing
dACS
oldLogE: -1545.9838094499692
oP
best (-1545.9838094499692, None)
found nothing
final LogE: -3300.395469673638
greedy LogE: -3300.395469673638
ICC 1: 	 101110100
ICC 2: 	 010001011

	 **** Filling in the rxr-triangular Matrix ****
	 **** Start Merging ****
	 **** Greedy Merging Finished ****

ICC 1: 	 010001011
ICC 2: 	 101110100



In [3]:
N = 100
pA = createPartition(N)
pB = createEmptyParition(N)

def pprint(p):
    print(p.array)
    # mod.Print_MCM_Partition(p)

pprint(pA)
pprint(pB)
swapVariable(pA, pB, 90)
pprint(pA)
pprint(pB)
swapVariable(pB, pA, 90)
pprint(pA)
pprint(pB)
print(bin(pA.array[0, 2]))
print(pA.array[0, 2].bit_count() + pA.array[0, 1].bit_count())
bin(8589934592)

[[                   0 18446744073709551615          68719476735]]
[[0 0 0]]
[[                   0 18446744073709551615          68652367871]]
[[       0        0 67108864]]
[[                   0 18446744073709551615          68719476735]]
[[0 0 0]]
0b111111111111111111111111111111111111
100


'0b1000000000000000000000000000000000'

In [2]:
mod.test()
r = 100
Kset = mod.read_datafile('MinCompSpin_Greedy/INPUT/my_data_n100_N1000.dat', r)
Partitions = createPartition(r)
logE = mod.LogE_ICC(Kset, Partitions)
assert logE != 0
print(logE)

ENTERING TEST !!!!!!!!!!!!!!
Read the dataset: MinCompSpin_Greedy/INPUT/my_data_n100_N1000.dat
Number of variables to read: n = 100
bitset count 100

TESTESTEST LogE: 0

Read the dataset: MinCompSpin_Greedy/INPUT/my_data_n100_N1000.dat
Number of variables to read: n = 100
LogE_ICC in cpp
ICC 1: 	 1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

bitset count 100


AssertionError: 